## LLM

In [1]:
import matplotlib.pyplot as plt
import random
from IPython.display import clear_output
from src.tokenizer import TokenizerBPE, TokenizerChar, word_split, normalize_to_ascii, pair_freq
from src.data_handling import read_first_n, normalize_to_ascii


import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
import numpy as np
import pickle as pkl
from tqdm.notebook import tqdm
from src.data_handling import fused_qa
import sys
import random
import json

# disable gpu for testing purposes
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [ ]:
"""
with open('corpus/triviaqa-rc/qa/web-train.json', 'r', encoding='utf-8') as f:
    squad = json.load(f)

question_web_list = []
answer_web_list = []
evidence_web_list = []

for qa in tqdm(squad['Data']):
    question_web_list.append(qa['Question'])
    answer_web_list.append(qa['Answer']["Value"])
    sr = qa['SearchResults']
    if len(sr) > 0: 
        evidence_web_list.append(sr[0]["Filename"])
    else:
        evidence_web_list.append(None)
"""

  0%|          | 0/76496 [00:00<?, ?it/s]

In [ ]:
"""

path = "corpus/triviaqa-rc/evidence/web/"

txt_list = []

for evidence in tqdm(evidence_web_list):
    if evidence is not None:
        txt = open(path + evidence, 'r', encoding='utf-8').read()
        txt_list.append(txt)
"""   

  0%|          | 0/76496 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
txt = "".join(txt_list)
print("Length of txt: ", len(txt))

Length of txt:  1163223076


In [2]:
with open('corpus/triviaqa-rc/qa/wikipedia-train.json', 'r', encoding='utf-8') as f:
    squad = json.load(f)

question_wiki_list = []
answer_wiki_list = []
evidence_wiki_list = []

for qa in tqdm(squad['Data']):
    question_wiki_list.append(qa['Question'])
    answer_wiki_list.append(qa['Answer']["Value"])
    sr = qa["EntityPages"]
    if len(sr) > 0: 
        evidence_wiki_list.append(sr[0]["Filename"])
    else:
        evidence_wiki_list.append(None)
    

  0%|          | 0/61888 [00:00<?, ?it/s]

In [4]:
path = "corpus/triviaqa-rc/evidence/wikipedia/"

evidence_article = {}
seen = {}

for i, evidence in tqdm(list(enumerate(evidence_wiki_list))):
    if evidence is not None:
        evidence = evidence.replace(":", "_").replace("?", "_").replace("*", "_").replace('"', "_")
        if not evidence in seen:
            seen[evidence] = 1
            try:
                txt = open(path + evidence, 'r', encoding='utf-8').read()
                evidence_article[evidence] = txt
            except:
                print("Error reading file: ", evidence)
                continue
        else:
            seen[evidence] += 1

  0%|          | 0/61888 [00:00<?, ?it/s]

In [5]:
rank = [[evidence, freq] for freq, evidence in sorted(zip(seen.values(), seen.keys()), reverse=True)]
print(len(rank))
print(rank[4990:5000])

28406
[['Blur_(band).txt', 3], ['Blott_on_the_Landscape.txt', 3], ['Bloody_Sunday_(1972).txt', 3], ['Bloody_Mary_(cocktail).txt', 3], ['Blepharitis.txt', 3], ['Blarney_Stone.txt', 3], ["Blake's_7.txt", 3], ['Blackcurrant.txt', 3], ['Black_hole.txt', 3], ['Black_Sea.txt', 3]]


In [6]:
def get_top_n_evidence(n, rank):
    top_n = []
    for i in range(n):
        article = evidence_article[rank[i][0]]
        top_n.append(article)
    return top_n

In [7]:
article_wiki_list = get_top_n_evidence(3000, rank)

text = "".join(article_wiki_list)
print("article_list: ", len(text))

article_list:  107693744


In [ ]:
text = "".join(question_wiki_list + answer_wiki_list)
print("Length of question_web_list: ", len(text))

Length of question_web_list:  5580683


In [ ]:
random.seed(42)

question_wiki_clean_list = [normalize_to_ascii(x) for x in question_wiki_list]
answer_wiki_clean_list = [normalize_to_ascii(x) for x in answer_wiki_list]
article_wiki_clean_list = [normalize_to_ascii(x) for x in article_wiki_list]

In [ ]:


corpus = question_wiki_clean_list + answer_wiki_clean_list + article_wiki_clean_list

#tokenizer = TokenizerChar(corpus)

tokenizer = TokenizerBPE(corpus, 16000, lowercase=True)
tokenizer.destroy_hash()

pkl.dump(tokenizer, open('tokenizers/trivia_16k.pkl', 'wb'))

Lowercasing corpus


  0%|          | 0/126776 [00:00<?, ?it/s]

Merging tokens


  0%|          | 0/16000 [00:00<?, ?it/s]

t h 2283821
i n 1808021
th e 1650954
a n 1517695
e r 1361619
o n 1221025
r e 1040645
a t 901993
e n 873632
e d 868777
o r 831623
a l 827763
s t 822449
a r 781239
o f 711807
an d 687112
i s 637624
a s 629339
e s 612191
i t 578174
i c 560089
in g 524059
t o 509951
r o 479139
i on 443019
l e 416757
o u 408293
c h 354605
en t 318918
i l 308665
s e 300449
a m 297264
e l 274825
c o 265617
a d 262858
w h 254693
a c 251229
b e 241116
f or 234044
l y 231240
d e 219450
u r 211655
u n 209864
o l 202883
i m 199800
i v 190917
i r 190665
w as 187359
i g 182565
at ion 179449
t er 173297
u s 172534
o w 172530
c on 170730
c e 168126
w i 161263
v er 156052
1 9 155803
b y 154206
a y 152396
i d 151675
m o 150050
a g 148737
c t 147787
h e 146947
s u 146654
m e 144022
' s 143450
wi th 142201
p e 142020
u l 131029
a re 130757
t e 130173
s o 130137
i st 129001
t r 127470
a p 127434
n e 124835
er s 122352
p o 120719
a v 120707
a b 120156
th at 119805
co m 118447
p l 115997
p ro 115509
al l 114412
re s 113258
a

In [20]:
def fused_qa(question_list, answer_list):
    q ="<q>"
    a = "<a>"
    sos = "<s>"
    eos = "</s>"

    corpus_list = []
    for question, answer in tqdm(list(zip(question_list, answer_list))):
        qa = [sos, q, question.lower(), a, answer.lower(), eos]
        corpus_list.extend(qa)
    
    return "".join(corpus_list)


def fused_article(corpus_list):
    sos = "<s>"
    eos = "</s>"

    corpus_padded_list = []
    for line in corpus_list:
        line = [sos, line.lower(), eos]
        corpus_padded_list.extend(line)
    
    return "".join(corpus_padded_list)

In [9]:
tokenizer = pkl.load(open('tokenizers/trivia_16k.pkl', 'rb'))
tokenizer.create_hash()

tokenizer.add_special_tokens(["<s>", "</s>", "<q>", "<a>",])

In [ ]:
corpus_qa_fused = fused_qa(question_wiki_clean_list, answer_wiki_clean_list)

corpus_qa_encoded = tokenizer.encode(corpus_qa_fused)

pkl.dump(corpus_qa_encoded, open('corpus/trivia_qa_16k_encoded', 'wb'))

  0%|          | 0/61888 [00:00<?, ?it/s]

In [ ]:
corpus_article_fused = fused_article(article_wiki_clean_list)

corpus_article_encoded = tokenizer.encode(corpus_article_fused)

pkl.dump(corpus_article_encoded, open('corpus/trivia_article_16k_encoded', 'wb'))

  0%|          | 0/16000 [00:00<?, ?it/s]

## 